# PageRank Algorithm
## Here we implement a (very) simple version of Google's PageRank Algorithm

### Import our libraries

In [6]:
from __future__ import division
import numpy as np
import scipy as sp
from scipy import sparse
import numpy.linalg as la

In [7]:
def to_matrix(filename, n):
    """Return the nxn adjacency matrix described by datafile.

    Parameters:
        datafile (str): The name of a .txt file describing a directed graph.
        Lines describing edges should have the form '<from node>\t<to node>\n'.
        The file may also include comments.
    n (int): The number of nodes in the graph described by datafile

    Returns:
        A SciPy sparse dok_matrix.
    """
    A = np.zeros((n,n))
    with open(filename, 'r') as myfile:
        for line in myfile:
            line = line.strip().split()
            try:
                A[int(line[0]),int(line[1])]=1
            except:
                continue
    return sparse.dok_matrix(A)


In [8]:
def calculateK(A,N):
    """Compute the matrix K as described in the lab.

    Parameters:
        A (ndarray): adjacency matrix of an array
        N (int): the datasize of the array

    Returns:
        K (ndarray)
    """
    if type(A) == sparse.dok_matrix:
        A = sparse.dok_matrix.toarray(A)
    #Fix A so that any rows of all zeros become all ones
    for i in xrange(len(A)):
        if np.all(A[i]==0):
            A[i]=1
            
    #Calculate D
    D = np.diag([A.sum(1)[i] for i in xrange(len(A))])
    #Return K
    return la.inv(D).dot(A).T

### First we sort them using an iterative method

In [9]:

def iter_solve(adj, N=None, d=.85, tol=1E-5):
    """Return the page ranks of the network described by 'adj'.
    Iterate through the PageRank algorithm until the error is less than 'tol'.

    Parameters:
        adj (ndarray): The adjacency matrix of a directed graph.
        N (int): Restrict the computation to the first 'N' nodes of the graph.
            If N is None (default), use the entire matrix.
        d (float): The damping factor, a float between 0 and 1.
        tol (float): Stop iterating when the change in approximations to the
            solution is less than 'tol'.

    Returns:
        The approximation to the steady state.
    """
    if type(adj) == sparse.dok_matrix:
        adj = sparse.dok_matrix.toarray(adj)
    if N==None:
        N = len(adj)

    K = np.array(calculateK(adj[:N,:N], N))
    pold = np.random.random(N)
    pold /= la.norm(pold)
    pnew = np.ones(N)
    i=0
    while la.norm(pnew - pold) > tol:
        pnew, pold = d*K.dot(pold) + (1-d)/N * np.ones(N), pnew
        #Iteration count
        i+=1
        if i > 500:
            raise ValueError('Did not converge')
    return pnew


### We can solve the same problem using eigenvalues

In [10]:
def eig_solve(adj, N=None, d=.85):
    """Return the page ranks of the network described by 'adj'. Use SciPy's
    eigenvalue solver to calculate the steady state of the PageRank algorithm

    Parameters:
        adj (ndarray): The adjacency matrix of a directed graph.
        N (int): Restrict the computation to the first 'N' nodes of the graph.
            If N is None (default), use the entire matrix.
        d (float): The damping factor, a float between 0 and 1.
        tol (float): Stop iterating when the change in approximations to the
            solution is less than 'tol'.

    Returns:
        The approximation to the steady state.
    """
    if type(adj) == sparse.dok_matrix:
        adj = sparse.dok_matrix.toarray(adj)
        
    if N==None:
        N = len(adj)

    K = calculateK(adj[:N,:N], N)
    E = np.ones((N,N))
    B = d*K + (1-d)/N * E

    eigvals = la.eig(B)[0]
    one_eigval = list(eigvals).index(max(eigvals))
    eigvecs = la.eig(B)[1]
    return np.real(eigvecs[:,one_eigval]/np.sum(eigvecs[:,one_eigval]))


### Now we'll use our iterative solver to organize some data about NCAA basketball teams in 2013, to rank them in order from best to worst.

In [13]:
def team_rank(filename='Data/ncaa2013.csv'):
    """Use iter_solve() to predict the rankings of the teams in the given
    dataset of games. The dataset should have two columns, representing
    winning and losing teams. Each row represents a game, with the winner on
    the left, loser on the right. Parse this data to create the adjacency
    matrix, and feed this into the solver to predict the team ranks.

    Parameters:
        filename (str): The name of the data file.
    Returns:
        ranks (list): The ranks of the teams from best to worst.
        teams (list): The names of the teams, also from best to worst.
    """
    all_teams=[]
    matches=[]
    #Get the data
    with open(filename, 'r') as ncaafile:
        ncaafile.readline() #reads and ignores the header line
        for line in ncaafile:
            team = line.strip().split(',') #split on commas
            all_teams.append(team[0])
            all_teams.append(team[1])
            matches.append(team)

    #Get the number of teams
    unique_teams = list(set(all_teams))
    team_count = len(unique_teams)

    #Here we're going to use (1) unique team name indices and (2) the game information
    # that we have, in order to make that adjacency matrix.
    A = np.zeros((team_count, team_count))

    for i in xrange(len(matches)):
        n, m = unique_teams.index(matches[i][0]), unique_teams.index(matches[i][1])
        A[m, n] = 1

    N = len(A)
    A1 = sparse.dok_matrix(A)
    rankings = iter_solve(A1, d=.7)
    teams = np.vstack((rankings, unique_teams)).T
    
    ranked_teams = teams[teams[:,0].argsort()][:,1][::-1]
    rankings = teams[teams[:,0].argsort()][:,0][::-1]
    return ranked_teams, rankings

# The team_rank function in action

In [14]:
print team_rank()

(array(['Duke', 'Butler', 'Louisville', 'Illinois', 'Indiana', 'Miami FL',
       'Syracuse', 'Ohio St', 'Michigan St', 'Kansas', 'Minnesota',
       'Michigan', 'Georgetown', 'Wisconsin', 'St Louis', 'Virginia',
       'New Mexico', 'Marquette', 'Notre Dame', 'NC State',
       'VA Commonwealth', 'Oklahoma St', 'Florida', 'Villanova', 'Temple',
       'Arizona', 'UNLV', 'Missouri', 'Cincinnati', 'Pittsburgh', 'Xavier',
       'UCLA', 'Connecticut', 'Colorado', 'Gonzaga', 'Creighton', 'Oregon',
       'San Diego St', 'California', 'Oklahoma', 'Wichita St', 'Maryland',
       'La Salle', 'Tennessee', 'Colorado St', 'North Carolina',
       'Kentucky', 'Indiana St', 'Mississippi', 'Boise St', 'Charlotte',
       'Iowa St', 'Baylor', 'Kansas St', 'Arkansas', 'Texas A&M', 'Iowa',
       'Georgia Tech', 'Air Force', 'Massachusetts', 'Florida St',
       'Memphis', 'Washington', 'Arizona St', 'USC', 'Wake Forest', 'Utah',
       'Alabama', "St John's", 'Northern Iowa', "St Mary's CA", 'Purdu